In [90]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 19
year = 2022

data = get_data(day=day, year=year)

# data = '\n'.join(re.split(r'[\.|:|\n]', data))
# data = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
# print(data)

In [83]:
data = """Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2:
  Each ore robot costs 2 ore.
  Each clay robot costs 3 ore.
  Each obsidian robot costs 3 ore and 8 clay.
  Each geode robot costs 3 ore and 12 obsidian.""".replace("\n  ", " ")

In [84]:
# bps = [[l.strip().split() for l in bp.split("\n")[1:]] for bp in data.split("\n\n")]

# bps = [[(l[1],list(zip(map(int, l[4::3]), l[5::3]))) for l in bp] for bp in bps]
# bps


2

In [33]:
import functools

global pruned, visited


In [114]:
def explore(minute, stocks, robots, costs, max_minutes):
    
    global visited
    
    if minute == max_minutes:
        return (stocks, robots)

    # Heuristic 1: aggregate any resource count that is above max ability to spend
    key_stocks = tuple(
        sys.maxsize if (s + (max_minutes-minute)*r > (max_minutes-minute)*max(c[material] for c in costs)) else s
        for material,(s,r) in enumerate(zip(stocks[:-1], robots[:-1]))
        ) + (stocks[3],)
    prev_min = visited.get((key_stocks, robots), max_minutes+1)
    if prev_min <= minute:
        return ((-1, -1, -1, -1), (0, 0, 0, 0))
    visited[(key_stocks, robots)] = minute
        
    make_robots = [(robot, robot_costs) for robot, robot_costs in reversed(list(enumerate(costs))) 
     if all(s>=c for s,c in zip(stocks, robot_costs))]

    # Heursistic 2: build geode crushers whenever possible
    if len(make_robots) > 0 and make_robots[0][0] == 3:
        make_robots = [r for r in make_robots if r[0] == 3]
    
    # Heuristic 3: if can build geode or obsidian, do not skip
    if len(make_robots) == 0 or make_robots[0][0] < 2:
        make_robots.append((4, (0,0,0)))
        
    solutions = [explore(
        minute=minute+1,
        stocks= tuple(s+r-c for s,r,c in zip(stocks, robots, robot_costs+(0,))),
        robots= tuple(r+1 if i == robot else r for i,r in enumerate(robots)),
        costs=costs,
        max_minutes=max_minutes)
                 for robot, robot_costs in make_robots]
    
    return max(solutions, key=lambda x:x[0][-1])

def explore_blueprint(bp, max_minutes):
    global visited
    visited = dict()

    bp_id, ore_c, clay_c, obs_ore_c, obs_clay_c, geodes_ore_c, geodes_obs_c = bp
    costs = ((ore_c, 0, 0), (clay_c, 0, 0), (obs_ore_c,
             obs_clay_c, 0), (geodes_ore_c, 0, geodes_obs_c))
    stocks, robots = explore(0, stocks=(0, 0, 0, 0), robots=(
        1, 0, 0, 0), costs=costs, max_minutes=max_minutes)

    # print("stocks:", stocks, "robots", robots, " | geodes: ", stocks[-1])

    return (bp_id, stocks[-1])


bps = [list(map(int, re.findall(r"(\d+)", l))) for l in data.split("\n")]

vals = [explore_blueprint(bp, max_minutes=24) for bp in bps]
print("Part 1:", sum(a * b for a, b in vals))

vals = [explore_blueprint(bp, max_minutes=32)[1] for bp in bps[:3]]
print("Part 2:", vals[0] * vals[1] * vals[2])


Part 1: 1599


In [48]:
print(sum((a+1)*b for a,b in vals))
# vals

# 1605 too high

126


In [74]:
vals[0][1] * vals[1][1] * vals[2][1]

# 12096 too low

# 14112 -> correct!

14976

In [68]:
49*18*16

12096

In [ ]:
# 1000000000000 
(1000000000000-37) // 35, (1000000000000-37) % 35

In [ ]:
Cycle starts: 37 bricks | height: 63
full cycle: 72 tot bricks | tot height: 116
cycle brick length: 35


In [ ]:
# min_flows = dict()
# # nx.bfs_tree(G, 'AA', depth_limit=30)
#     # print(t)
# cur_states = {('AA', (), 0)}
# tot_minutes = 30

# for minute in range(1, tot_minutes+1):
#     next_states = set()
#     for cur_node, open_valves, flow in cur_states:
#         flow_rate = sum(flows[n] for n in open_valves)
#         min_flow = flow + flow_rate * (tot_minutes-minute)
#         # print(f"Exploring: {cur_node} {open_valves} {flow}")
#         if min_flows.get(cur_node, -1) >= min_flow:
#             # print(f" -> Pruning: {flow} + {tot_minutes-minute} * {flow_rate} -> {min_flow} <= {min_flows[cur_node]}")
#             continue
#         # print(f" Adding min_flow: {cur_node} -> {min_flow}")
#         min_flows[cur_node] = min_flow
#         flow += flow_rate
#         if (cur_node not in open_valves) and flows[cur_node]:
#             next_states.add((cur_node, tuple(set([*open_valves, cur_node])), flow))
#         for next_node in G.neighbors(cur_node):
#             next_states.add((next_node, open_valves, flow))
#     if len(next_states) == 0:
#         pprint(cur_states)
#         pprint(min_flows)
#     print(f"Minute {minute}: max flow = {max(f for _,_,f in next_states)} ({len(next_states)} active states)")
#     cur_states = next_states
    
#     # pprint(min_flows)

In [ ]:
# sum(v for k,v in flows.items() if k in ('BB', 'DD', 'HH'))

    # ('BB', 'HH') ('DD', 'JJ
# flows

In [ ]:
all_valves = {k for k,v in flows.items() if v}
min_flows = dict()
# nx.bfs_tree(G, 'AA', depth_limit=30)
    # print(t)
cur_states = {(('AA', 'AA'), (), 0, ())}
tot_minutes = 26

for minute in range(1, tot_minutes+1):
    print(f"Minute {minute}: max flow = {max([0, *min_flows.values()])} ({len(cur_states)} active states)\n")
        
    next_states = set()
    for cur_nodes, open_valves, flow, prev_nodes in  sorted(cur_states, key=lambda x: x[2], reverse=True):
        flow_rate = sum(flows[n] for n in open_valves)
        flow += flow_rate
        
        min_flow = flow + flow_rate * (tot_minutes-minute)

        if min_flows.get(cur_nodes, -1) >= min_flow:
            continue
        
        min_flows[cur_nodes] = min_flow
        
        if len(open_valves) == len(all_valves):
            continue
        
            
        next_states_n = [[], []]
        for i,cur_node in enumerate(cur_nodes):
            next_states_n[i] = [([(next_node, []) 
                                 for next_node in G.neighbors(cur_node)
                                 if next_node not in (prev_nodes)]
                                + ([(cur_node, [cur_node])] if flows[cur_node] and cur_node not in open_valves else []))
                                
            ]
        debug = {(
            tuple(sorted([cur_node1, cur_node2])), 
            tuple(sorted(set([*valve1, *valve2, *open_valves]))), 
            flow,
            (cur_nodes)
        )
                            for cur_node1, valve1 in next_states_n[0]
                            for cur_node2, valve2 in next_states_n[1]
                            }

        next_states.update(debug)
        
    cur_states = next_states

    if len(cur_states) == 0:
        print(f"No more moves.")
        break
    
print(f" Max flow: {max(min_flows.values())}")
    # pprint(min_flows)
    # print(max(min_flows.values()), '\n')
    # 2532
    # 2349
    # 2715
    
    # ('BB', 'HH') ('DD', 'JJ

In [ ]:
max(min_flows.values())

In [ ]:
print(f"\nMinute {minute}: max flow = {max(f+sum(flows[n] for n in o) for _,o,f in cur_states)} ({len(cur_states)} active states)\n")

In [ ]:
p, v, f, _ = max(cur_states, key=lambda x:x[2])

In [ ]:
len(v)

In [ ]:
sum(f > 0 for f in flows.values())